<h2>3D Analysis</h2>

Let’s run the 3D Vessel Analysis

Parameter Selection:

<b>Features selection</b> Classes to include in the statistical analysis :<br><br>
   <b>String vessel</b>: String vessel class.<br>
   <b>Normal vessel</b>: Normal vessel class.<br>
   <b>Both (all)</b>: Global structure String vessel union Normal vessel.<br><br>

For the <b>Vessel Analysis Parameters</b>:<br><br>

Please ensure that the Pixel Dimension is set correctly.<br>

For all other parameters, please refer to the [paper](https://www.cell.com/cell-reports-methods/pdf/S2667-2375(23)00051-6.pdf), specifically Table 1 on page 4.<br><br>

<b>Input/Output Paths</b>:<br><br>

   <b>Input Path</b>:<br>
   This refers to the location of your segmentation results provided to the model.<br>

   <b>Output Path</b>:<br> 
   In this analysis, one or two sets of “raw” statistics will be generated for each file. If valid string vessels are detected, two statistics files will be produced: one for all vessels and one specifically for string vessels. If no valid string vessels are detected, only one statistics file for all vessels will be generated. Additionally, a summary CSV file will be created. Please specify the path to save the “raw” statistics as well as the filename for the summary CSV.

  

In [ ]:
from core.analysis import analysis_menu
analysis_menu()

<h2>Quality Control Analysis</h2>

After generate the csv files with the statistical metrics just select the path to the data_full_stats.csv file and execute the pipeline. 

In [ ]:
from core.analysis import qc_menu
qc_menu()